In [1]:
import pandas as pd
import numpy as np

In [2]:
data_filepath = r"C:\Users\emanlangit\Documents\2025\00 ACSIN\New Data\SCM_Data_21-02-25_Rev_1.28_input.csv"
Jan_test_df = pd.read_csv(data_filepath, delimiter = ",", low_memory=False)

In [4]:
data_filepath = r"C:\Users\emanlangit.AEONCREDIT\OneDrive - AEON Credit Service Systems (Philippines), Inc\Desktop\misc\Copy of Test Cases New_Jan'24Feb'24_Rev.csv"
Dec_test_df = pd.read_csv(data_filepath, delimiter = ",", low_memory=False)

In [6]:
Dec_test_df.columns

Index(['bureau_score', 'down_payment_amount', 'branch_name',
       'disbursal_amount', 'requested_loan_amount', 'loan_asset_cost',
       'final_sanctioned_amount', 'original_emi_amount', 'age',
       'number_of_installments', 'res_city', 'res_state', 'total_income',
       'product_description', 'industry', 'marital_status', 'manufacturer',
       'occupation_type', 'res_resident_status', 'loan_account_no',
       'loan_application_date', 'D3_within12months'],
      dtype='object')

product_description
TWO-WHEELER    8885
Name: count, dtype: int64

In [8]:
Jan_test_df.columns

Index(['loan_account_no', 'APPLICATION_NUMBER', 'CIFNUMBER', 'CUSTOMER_NUMBER',
       'Application Received Date', 'product_description', 'Judgement date',
       'DECISION', 'USER_COMMENT', 'LOAN_ACCOUNT_NO_og',
       'LOAN_SANCTIONED_DATE', 'LOAN_AMOUNT', 'DISBURSAL_DATE',
       'LOAN_ASSET_COST', 'number_of_installments', 'Channel',
       'ORIGINAL_EMI_AMOUNT', 'interest_rate', 'res_Years_at_Current_City',
       'Years_in_Occupation', 'res_Years_at_Current_Address',
       'res_Resident_Status', 'res_City', 'res_State', 'AGE', 'DATE_OF_BIRTH',
       'Gender', 'Manufacturer', 'Down_Payment_Amount', 'Asset_Model',
       'Dealer', 'Total_Income', 'Branch_Name', 'Occupation_Type', 'Industry',
       'Marital_Status', 'Nature_Of_Business', 'Loan Amount Requested',
       'bureau_score', 'D3_within12months'],
      dtype='object')

In [26]:
Jan_test_df.shape
Jan_reduced = Jan_test_df.copy()

In [29]:
Jan_test_df.product_description.value_counts()

product_description
2 Wheeler Loan           17484
Used Two Wheeler Loan     1117
E - Bike                    14
Used Car                     8
Name: count, dtype: int64

In [36]:
Jan_reduced1 = Jan_reduced[Jan_reduced.product_description == "2 Wheeler Loan"]
Jan_reduced1.shape

(17484, 40)

In [37]:
non_TW = 1117 + 14 +8
print(non_TW)

1139


In [42]:
Jan_reduced1['LOAN_ACCOUNT_NO_og'].isnull().sum()

6760

In [43]:
17484 - 6760

10724

In [44]:
Jan_reduced2 = Jan_reduced1.dropna(subset = ['LOAN_ACCOUNT_NO_og'])
Jan_reduced2.shape

(10724, 40)

In [45]:
duplicated_A = Jan_reduced2.duplicated(subset='LOAN_ACCOUNT_NO_og', keep =False)
duplicated_A.sum()

In [48]:
Jan_reduced3 = Jan_reduced2[~duplicated_A]

In [49]:
Jan_reduced3.shape

(6928, 40)

In [64]:
Jan_reduced3.LOAN_ACCOUNT_NO_og.unique()

array(['TW01010101000002287379', 'TW01010107000002289052',
       'TW01010106000002290584', ..., 'TW01010103000002288457',
       'TW01010103000002292968', 'TW01010105000002306301'], dtype=object)

In [101]:
Jan_LANs = set(Jan_reduced3.LOAN_ACCOUNT_NO_og.unique())
Jan_LANs_dupped = set(Jan_test_df.LOAN_ACCOUNT_NO_og.unique())

### End of Jan Test screening

### Start of Dec Test screening

In [47]:
Dec_test_df.shape

(8885, 22)

In [50]:
Dec_test_df.product_description.value_counts()

product_description
TWO-WHEELER    8885
Name: count, dtype: int64

In [52]:
Dec_test_df['loan_account_no'].isnull().sum()

0

In [53]:
Dec_test_df['loan_account_no'].nunique()

8885

In [54]:
duplicated_B = Dec_test_df.duplicated(subset='loan_account_no', keep =False)
duplicated_B.sum()

0

In [13]:
#duplicates = Jan_test_df[duplicated_A] #1915 duplicated LANs

In [71]:
Dec_LANs = set(Dec_test_df.loan_account_no.unique())

In [102]:
Dec_unique_LAN = Dec_LANs - Jan_LANs
Dec_unique_LAN_dupped = Dec_LANs - Jan_LANs_dupped
Jan_unique_LAN = Jan_LANs - Dec_LANs
Jan_unique_LAN_dupped = Jan_LANs_dupped - Dec_LANs
unique_LANs_intersection = Dec_LANs.intersection(Jan_LANs)
dupped_LANs_intersection = Dec_LANs.intersection(Jan_LANs_dupped)

In [103]:
print(f'unique: {len(unique_LANs_intersection)}')
print(f'duplicated: {len(dupped_LANs_intersection)}')

unique: 6587
duplicated: 8154


In [79]:
set1 = set(["a","b","c","d","e"])
set2 = set(["a","d","e","f","g","h"])

s1_unique = set1 - set2
s2_unique = set2 - set1

In [80]:
s2_unique

{'f', 'g', 'h'}

#duplicates removed
Dec_LANs minus Jan_LANs = 2298 #unique LANs in December File
Jan_LANs minus Dec_LANs = 341 #unique LANs in January File

#with duplicates
Jan_LANs_dupped - Dec_LANs = 1371 #unique LANs in January File

In [96]:
len(Dec_unique_LAN_dupped)

731

## CIBIL score distribution comparison

In [59]:
#dfs to compare PSI
working_df1 = Dec_test_df.copy()
working_df2 = Jan_reduced3.copy()

In [60]:
print(working_df1.shape)
print(working_df2.shape)

(8885, 22)
(6928, 40)


In [61]:
#bureau_score
def bureau_score_bin(x):
    try:
        y = float(x)
        
        if y >= 750:
            return 'g [750, Inf)'
        elif y >= 731 and y <= 749:
            return 'e [731,749]'
        elif y >= 700 and y <= 730:
            return 'd [700,730]'
        elif y >= 680 and y <= 699:
            return 'c [680,699]'
        elif y >= 300 and y <= 679:
            return 'b [300,679]'
        elif y < 300:
            return 'a NTC'        
    except:
        return x

working_df1.loc[:, 'bureau_score_bin'] = working_df1['bureau_score'].apply(bureau_score_bin)
t1_bureau_score = working_df1['bureau_score_bin'].value_counts().sort_index()

working_df2.loc[:, 'bureau_score_bin'] = working_df2['bureau_score'].apply(bureau_score_bin)
t2_bureau_score = working_df2['bureau_score_bin'].value_counts().sort_index()

bureau_score_df = pd.DataFrame({f'Below 300':t1_bureau_score, 
                           f'300-749':t2_bureau_score})

In [62]:
bureau_score_df

,Below 300,300-749
bureau_score_bin,,
a NTC,3354,2774
"b [300,679]",433,324
"c [680,699]",366,271
"d [700,730]",1350,1057
"e [731,749]",1280,954
"g [750, Inf)",2102,1548


## Duplicated Rows analysis

In [87]:
Jan_test_df.shape

(18623, 40)

In [88]:
duplicated_A = Jan_reduced2.duplicated(subset='LOAN_ACCOUNT_NO_og', keep =False)

In [89]:
Jan_duplicates = Jan_reduced2[duplicated_A]
Jan_duplicates.shape

(3796, 40)

In [90]:
Jan_duplicates

,loan_account_no,APPLICATION_NUMBER,CIFNUMBER,CUSTOMER_NUMBER,Application Received Date,product_description,Judgement date,DECISION,USER_COMMENT,LOAN_ACCOUNT_NO_og,...,Dealer,Total_Income,Branch_Name,Occupation_Type,Industry,Marital_Status,Nature_Of_Business,Loan Amount Requested,bureau_score,D3_within12months
2,TEST3,APPL00288662,GLBCUST00000249888,CUST00258359,2/26/2024,2 Wheeler Loan,2/26/2024,Approve,TEST PROGRAM FOR CD TW,TW01020102000002296182,...,Sri Renuka Motors - Bangalore,516000,South Bangalore,Salaried,Others,Married,Services,109000,-1,0
3,TEST4,APPL00289273,GLBCUST00000250403,CUST00258951,2/28/2024,2 Wheeler Loan,3/19/2024,Approve,TEST PROGRAM FOR CD TW | SCHEME CHANGE DOEN,TW01020101000002299657,...,Ekadantha Motors - East Bangalore,348000,East Bangalore,Salaried,Others,Single,Services,97000,-1,0
5,TEST6,APPL00285112,GLBCUST00000246883,CUST00255022,2/16/2024,2 Wheeler Loan,2/26/2024,Approve,No test code DEVIATION APPROVAL FROM PRANALI M...,TW01010102000002295981,...,KHUSHI AUTOMOBILES - GHANSOLI,360000,Harbour Mumbai,Salaried,Others,Single,Services,86000,638,0
9,TEST10,APPL00269683,GLBCUST00000233494,CUST00241067,1/7/2024,2 Wheeler Loan,1/12/2024,Approve,no test code,TW01030104000002288295,...,Shri Yash Motors - Noida,360000,Noida,Salaried,Apparels,Married,Others,83000,-1,0
11,TEST12,APPL00278457,GLBCUST00000241145,CUST00248971,1/30/2024,2 Wheeler Loan,2/1/2024,Approve,NO TEST CODE,TW01030101000002292111,...,JSB Auto Cars - Gudgaon,360000,Gurgaon,Salaried,Others,Married,Services,76000,721,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17695,TEST17696,APPL00288335,GLBCUST00000249611,CUST00258045,2/25/2024,2 Wheeler Loan,2/25/2024,Approve,no test code,TW01010106000002295822,...,BIKE CLINIC - VASAI ROAD,420000,Virar,Self Employed Non Professional,Others,Married,Others,80500,778,0
17697,TEST17698,APPL00285425,GLBCUST00000247162,CUST00255308,2/17/2024,2 Wheeler Loan,2/23/2024,Approve,NO TEST CODE,TW01030107000002297829,...,Globe Autowheels-Delhi,450000,Ghaziabad,Self Employed Non Professional,Others,Married,Others,85000,-1,1
17698,TEST17699,APPL00281663,GLBCUST00000243896,CUST00251871,2/7/2024,2 Wheeler Loan,2/17/2024,Approve,TEST PROGRAM FOR CD-TW,TW01030104000002297851,...,Shri Yash Motors - Noida,360000,Noida,Self Employed Non Professional,Others,Single,Manufacturing,53300,-1,0
17700,TEST17701,APPL00278308,GLBCUST00000241007,CUST00248829,1/29/2024,2 Wheeler Loan,1/31/2024,Approve,NO TEST CIDE - SCHEME CHANGED,TW01020102000002291500,...,MURALI AUTOMOBILES - BANGALORE,468000,South Bangalore,Salaried,Construction,Single,Services,112000,734,0


In [93]:
# Group by LOAN_ACCOUNT_NO_og and check which columns have different values
def compare_duplicates(group):
    if len(group) > 1:
        # Get columns that have different values within the group
        diff_cols = []
        for col in group.columns:
            if not (group[col] == group[col].iloc[0]).all():
                diff_cols.append(col)
        return diff_cols

# Apply the comparison to each group of duplicates
differences = Jan_duplicates.groupby('LOAN_ACCOUNT_NO_og').apply(compare_duplicates)

# Print the results
# print("Columns that differ for each duplicated LOAN_ACCOUNT_NO_og:")
# for loan_account, diff_cols in differences.items():
#     if diff_cols:  # Only show if there are differences
#         print(f"\nLoan Account: {loan_account}")
#         print(f"Different columns: {diff_cols}")

# Optional: Get a summary of which columns most commonly differ
all_diff_cols = []
for diff_cols in differences:
    if diff_cols:
        all_diff_cols.extend(diff_cols)

diff_cols_summary = pd.Series(all_diff_cols).value_counts()
print("\nMost common differing columns:")
print(diff_cols_summary)


Most common differing columns:
loan_account_no              1897
res_Years_at_Current_City    1792
res_Resident_Status           423
res_City                       28
res_State                      12
Nature_Of_Business              8
Industry                        2
USER_COMMENT                    1
Name: count, dtype: int64


C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_5248\2675878076.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  differences = Jan_duplicates.groupby('LOAN_ACCOUNT_NO_og').apply(compare_duplicates)


In [105]:
differences

LOAN_ACCOUNT_NO_og
TW01010101000002287278         [loan_account_no, res_Years_at_Current_City]
TW01010101000002287281         [loan_account_no, res_Years_at_Current_City]
TW01010101000002287395         [loan_account_no, res_Years_at_Current_City]
TW01010101000002287436         [loan_account_no, res_Years_at_Current_City]
TW01010101000002287798         [loan_account_no, res_Years_at_Current_City]
                                                ...                        
TW01030108000002294033    [loan_account_no, res_Years_at_Current_City, r...
TW01030108000002294655    [loan_account_no, res_Years_at_Current_City, r...
TW01030108000002295231    [loan_account_no, res_Years_at_Current_City, r...
TW01030108000002295671         [loan_account_no, res_Years_at_Current_City]
TW01030108000002295859         [loan_account_no, res_Years_at_Current_City]
Length: 1897, dtype: object

In [115]:
Jan_duplicates[['LOAN_ACCOUNT_NO_og', 'res_Years_at_Current_City']].sort_values(by='LOAN_ACCOUNT_NO_og')[:25]

,LOAN_ACCOUNT_NO_og,res_Years_at_Current_City
17630,TW01010101000002287278,22
1357,TW01010101000002287278,0
17631,TW01010101000002287281,22
1358,TW01010101000002287281,0
12978,TW01010101000002287395,0
8275,TW01010101000002287395,23
16922,TW01010101000002287436,0
12304,TW01010101000002287436,22
673,TW01010101000002287798,22
16932,TW01010101000002287798,0


In [113]:
Jan_duplicates[['LOAN_ACCOUNT_NO_og', 'res_Resident_Status']].sort_values(by='LOAN_ACCOUNT_NO_og')

,LOAN_ACCOUNT_NO_og,res_Resident_Status
17630,TW01010101000002287278,Owned by Parent /Sibling
1357,TW01010101000002287278,Owned by Parent /Sibling
17631,TW01010101000002287281,Owned by Parent /Sibling
1358,TW01010101000002287281,Owned by Parent /Sibling
12978,TW01010101000002287395,Owned by Parent /Sibling
...,...,...
12812,TW01030108000002295231,Owned by Parent /Sibling
16331,TW01030108000002295671,Rented
11676,TW01030108000002295671,Rented
398,TW01030108000002295859,Owned by Self/Spouse


In [114]:
# Group by LOAN_ACCOUNT_NO_og and filter groups where res_Resident_Status differs
different_status = (Jan_duplicates
    .groupby('LOAN_ACCOUNT_NO_og')['res_Resident_Status']
    .agg(lambda x: len(set(x)) > 1)  # True if there are different values
    .reset_index()
)

# Filter the original dataframe to show only those groups
different_status_records = Jan_duplicates[
    Jan_duplicates['LOAN_ACCOUNT_NO_og'].isin(
        different_status[different_status['res_Resident_Status']]['LOAN_ACCOUNT_NO_og']
    )
][['LOAN_ACCOUNT_NO_og', 'res_Resident_Status']].sort_values('LOAN_ACCOUNT_NO_og')

different_status_records

,LOAN_ACCOUNT_NO_og,res_Resident_Status
2720,TW01010101000002290604,Owned by Parent /Sibling
16697,TW01010101000002290604,Rented
2893,TW01010101000002293916,Owned by Self/Spouse
2894,TW01010101000002293916,Owned by Parent /Sibling
10194,TW01010101000002294066,Owned by Parent /Sibling
...,...,...
7809,TW01030108000002294033,Rented
3185,TW01030108000002294655,Owned by Parent /Sibling
5546,TW01030108000002294655,Rented
3453,TW01030108000002295231,Rented


In [118]:
# Group by LOAN_ACCOUNT_NO_og and filter groups where res_city differs
different_status = (Jan_duplicates
    .groupby('LOAN_ACCOUNT_NO_og')['res_City']
    .agg(lambda x: len(set(x)) > 1)  # True if there are different values
    .reset_index()
)

# Filter the original dataframe to show only those groups
different_status_records = Jan_duplicates[
    Jan_duplicates['LOAN_ACCOUNT_NO_og'].isin(
        different_status[different_status['res_City']]['LOAN_ACCOUNT_NO_og']
    )
][['LOAN_ACCOUNT_NO_og', 'res_City']].sort_values('LOAN_ACCOUNT_NO_og')

different_status_records

,LOAN_ACCOUNT_NO_og,res_City
7680,TW01020102000002287270,BANGALORE
14708,TW01020102000002287270,DAKSHINA KANNADA
17677,TW01020102000002300868,CHENNAI
17676,TW01020102000002300868,BANGALORE
1334,TW01030104000002289675,GAUTAM BUDDHA NAGAR
17598,TW01030104000002289675,GHAZIABAD
4748,TW01030104000002289758,GAUTAM BUDDHA NAGAR
16408,TW01030104000002289758,GHAZIABAD
12626,TW01030104000002289777,DELHI
5633,TW01030104000002289777,GHAZIABAD
